In [1]:
import pandas as pd
from typing import Optional, List
from core import system_msg, user_msg, assistant_msg, generic_call_stream
from qdrant_search import QdrantSearchEngine

In [2]:
from psql_models import get_session

In [3]:
engine = QdrantSearchEngine()

In [4]:
#await engine.initialize()
await engine.sync_from_postgres(get_session)

Created text index for field: topic_name
Created text index for field: summary
Created text index for field: details


In [5]:
pd.options.display.max_colwidth = 1000

In [6]:
class ChatSession:
    def __init__(self, search_engine: QdrantSearchEngine, prompts):
        self.search_engine = search_engine
        self.prompts = prompts
        self.messages: List = []
        self.results = None
    
    def add_message(self, role: str, content: str):
        """Add a message to the conversation history"""
        if role == "system":
            self.messages.append(system_msg(content))
        elif role == "user":
            self.messages.append(user_msg(content))
        elif role == "assistant":
            self.messages.append(assistant_msg(content))
    
    async def process_user_message(self, query: str):
        """Process a user message and generate a response"""
        self.add_message("user", query)
        
        # Split query and instructions if present
        query, instructions = query.split(':') if ':' in query else (query, '')
        
        # Search for context using both methods
        main_results = await self.search_engine.search(
            query_text=query,
            limit=200,
            min_score=0.4,
        )
        
        speaker_results = await self.search_engine.search_by_speaker(
            speaker_query=query,
            limit=200,
            min_score=0.49
        )
        
        # Convert results to DataFrames
        main_df = pd.DataFrame(main_results) if main_results else pd.DataFrame()
        speaker_df = pd.DataFrame(speaker_results) if speaker_results else pd.DataFrame()
        
        # Select relevant columns and combine results
        columns = ['topic_name', 'speaker_name', 'summary', 'details']
        score_columns = ['score', 'vector_scores', 'exact_matches']
        
        if len(main_df) > 0:
            main_df = main_df[columns + score_columns]
            main_df['source'] = 'main'
        else:
            main_df = pd.DataFrame(columns=columns + score_columns + ['source'])
        
        if len(speaker_df) > 0:
            speaker_df = speaker_df[columns + ['score']]  # Speaker search has simpler scoring
            speaker_df['source'] = 'speaker'
        else:
            speaker_df = pd.DataFrame(columns=columns + ['score', 'source'])
        
        # Combine, deduplicate and sort results
        self.results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)
        if not self.results.empty:
            self.results = self.results.sort_values('score', ascending=False)

        # Create context from results
        context = self.results[columns].to_markdown(index=False) if not self.results.empty else "No relevant context found."
        
        # Add context message
        context_msg = system_msg(f"Context: {context}")
        
        # Generate response
        messages_with_context = [
            system_msg(self.prompts.perplexity),
            system_msg('special instructions: ' + instructions),
            *self.messages,
            context_msg
        ]
        
        response = await generic_call_stream(messages_with_context)
        
        # Add assistant response
        self.add_message("assistant", response)
        
        return response

    def get_results(self) -> Optional[pd.DataFrame]:
        """Get the latest search results"""
        return self.results

In [7]:
# Initialize
from prompts import Prompts
prompts = Prompts()
search_engine = QdrantSearchEngine()
chat = ChatSession(search_engine, prompts)

In [8]:

# Process a message
response = await chat.process_user_message("alexey")

Alexey is involved in several key aspects of project development, particularly in relation to user experience (UX) and technical solutions. He is responsible for providing a new build aimed at resolving issues with the extension's functionality, specifically addressing problems with buttons and overall performance[1][6]. Additionally, Alexey collaborates on the design elements of the project, focusing on the UX and dashboard design[2][3].

While currently on vacation, Alexey has also contributed to onboarding processes by releasing a project component related to this area[3]. His expertise is recognized in the Web3 space, where he is seen as a valuable contact for insights into audience preferences[4]. Furthermore, there are plans to leverage his network for promoting the project, indicating his influential role within the team[5].


In [9]:
from IPython.display import Markdown
Markdown(response)

Alexey is involved in several key aspects of project development, particularly in relation to user experience (UX) and technical solutions. He is responsible for providing a new build aimed at resolving issues with the extension's functionality, specifically addressing problems with buttons and overall performance[1][6]. Additionally, Alexey collaborates on the design elements of the project, focusing on the UX and dashboard design[2][3].

While currently on vacation, Alexey has also contributed to onboarding processes by releasing a project component related to this area[3]. His expertise is recognized in the Web3 space, where he is seen as a valuable contact for insights into audience preferences[4]. Furthermore, there are plans to leverage his network for promoting the project, indicating his influential role within the team[5].

In [10]:
chat.results[chat.results['source']=='speaker']

,topic_name,speaker_name,summary,details,score,vector_scores,exact_matches,source
99,Курсор,Alex Shevliakov,"Курсор is a code editor that Alex has recently started using, which offers improved functionality over VS Code.","Alex discusses the benefits of Курсор, particularly its ability to enhance coding efficiency and streamline the development process.",0.592849,NaN,NaN,speaker
96,Collaboration and support among team members,Alex Shevliakov,"The need for better collaboration and support among team members was discussed, particularly in terms of sharing knowledge and resources.",AS and DG agreed that sharing insights and providing support to each other could enhance productivity and project outcomes. They expressed a willingness to assist each other with coding and design tasks as needed.,0.592849,NaN,NaN,speaker
97,VS Code,Alex Shevliakov,VS Code is a popular code editor that Alex and Dmitry discuss in relation to their development work.,"Alex shares his experience with VS Code and how it compares to other tools like Курсор, emphasizing the importance of efficient coding tools in their workflow.",0.592849,NaN,NaN,speaker
98,Payment Methods and Billing Integration,Alex Shevliakov,Integrate payment methods and billing options into the platform to facilitate user transactions.,Consider using Stripe for payment processing and ensure users can manage their payment methods and view invoices easily.,0.592849,NaN,NaN,speaker
103,Astra,Alex Shevliakov,Astra is mentioned as another modern framework that could enhance development practices.,Alex advocates for using modern technologies like Astra to streamline the development process.,0.592849,NaN,NaN,speaker
100,dashboard,Alex Shevliakov,The dashboard is a component of the project that Alex is tasked with developing to match design specifications.,"Alex aims to ensure the dashboard is visually appealing and functional, while managing server logic separately.",0.592849,NaN,NaN,speaker
101,Update old buttons and banners,Alex Shevliakov,The team agreed to replace outdated buttons and banners to modernize the interface.,"Alex confirmed that he would take responsibility for updating the old buttons and banners. A timeline for completion was not specified, but it is a priority task.",0.592849,NaN,NaN,speaker
102,CRM,Alex Shevliakov,CRM refers to customer relationship management tools that could benefit from the discussed assistant features.,Alex suggests that the assistant could integrate with a minimalistic CRM to enhance user interactions and follow-up processes.,0.592849,NaN,NaN,speaker
94,Logo and Branding Elements,Alex Shevliakov,The need for a logo and branding elements was discussed to enhance the product's identity.,Alex will look into creating a logo and branding elements that align with the product's design and marketing strategy.,0.592849,NaN,NaN,speaker
95,Берлин,Alex Shevliakov,Berlin is mentioned as a travel destination in the context of a trip taken by Alex.,"Alex traveled from Munich to Berlin, highlighting his recent travels.",0.592849,NaN,NaN,speaker
